## CS 559 Homework 4

### Nan Chen

### SID: 20018997

### Email: nchen15@stevens.edu

### 1 Neural Network Backward Propagation [50 pts]
Consider neural networks with one hidden layer, as discussed during the lecture. The forward
propagation prediction was implemented. Complete the backward propagation codes. Using the
generated data, predict y.
```
x, y = make regression(n samples=100, n features=5, noise=10, random state=42)
```

### Forward Propagation
* The linaer combination from the input layer: $ A=\sum W_1X $
* The nonlinear activation function: $\sigma(A)=\frac{1}{1+e^{-A}}$
* The hidden layer output $Z$: $Z=\sigma(A)$
* The output layer output $\hat{Y}$: $\hat{Y} = \sigma(W_2 Z)$
* The error function: $E=\frac{1}{2}\sum (\hat{Y}-Y)^2$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

x, y = make_regression(n_samples=100, n_features=5, noise=10, random_state=42)

w1 = np.random.rand(50,5)
w2 = np.random.rand(1,50)

def ffNN(w,x):
    a = w.dot(x.T)
    return a

def sigmoid(x):
    return 1/(1+np.exp(-x))

def _delta(y,ypred):
    return ypred-y

def _cost(y,ypred):
    cost = np.sum(_delta(y,ypred)**2)/(2*len(y))
    return cost

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

## input layer to hidden layer
a1 = 
z1 = 
print(f'shape of a1: {a1.shape}, z1: {z1.shape}')

## hidden layer to output layer
yhat = 
print(f'shape of y1: {yhat.shape}')
print(f'cost: {_cost(y,yhat)}')

### Error Backpropagation: Output layer to hidden layer
a. (15 pts) 
* The error on output layer: $\delta_2 = \hat{Y}-Y$ where $\hat{Y}$ is the predicted value. 
* The dradient of Error: $\nabla E = \frac{\partial E}{\partial W_2} = \sum\delta_2 Z$
* The update $W_2$ value: $\hat{W}_2 = W_2 + \Delta W_2 = W_2 - \eta\nabla E$
* Compute $\delta_2$ and $\frac{\partial E}{\partial W_2}$ . Then, update $W_2$. 
* Use the learning rate $\eta = 1*10^{-4}$

In [ ]:
delta2 = 

dE_dW2 = 

eta = 0.0001
delta_w2 = 
w2_new = 

### Error Backpropagation: Hidden layer to input layer
b. (15 pts)
* The error on the hidden layer: $\delta_1 = \sigma'(A)\sum\delta_2 W_2$ where $\sigma'(A) = Z[1-Z]$.
* The update of $W_1$: $\hat{W}_1 = W_1 + \Delta W_1 = W_1 - \eta\delta_1 X$ 
* Compute $\delta_1$ and $\frac{\partial E}{\partial W_1}$ . Then, update $W_1$. 
* Use the learning rate $\eta = 1*10^{-4}$

delta1 = 
dz1 = 
delta1 = 

delta_w1 = 
w1_new = w1-eta*delta_w1

## Predict with new forward propagation
a1 = 
z1 = 

## hidden layer to output layer
yhat = 

### Prediction with updated $W_1$ and $W_2$:

### Put all together
c. (15 pts)
* Put the forward and backward propagation codes together in a for-loop to observe the error convergence.

In [ ]:
def fBNN(x,y,w1,w2,eta,epoch):
    cost_list = []
    epoch_list = []
    ypred_list = []
    for i in range(1,epoch+1):
        """
        forward propagation
        """

        """
        backpropagation from the output layer to the hidden layer
        """

        """
        backpropagation from the hidden layer to the input layer
        """        



        """
        forward propagation with new weights
        """

        
        if i>=1 and cost_new >=5E-3:
            epoch_list.append(i)
            cost_list.append(cost_new)
            ypred_list.append(yhat_new)
        if i > 1 and cost_new <= 5E-3:
            print("last epoch:",i,"cost:",cost,"y_pred:",yhat_new)
            break
    print(cost_new)      
    plt.plot(epoch_list, cost_list)
    plt.xlabel('epoch')
    plt.ylabel('cost')
    plt.show()

d. (5 pts)
* Use Scikit-learn Neural Network to predict the target variable.

In [ ]:
fBNN( x= , y = , w1 = , w2 = , eta = , epoch =)